In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Closed Book QA
t5_qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-large-ssm-nq")
t5_tok = AutoTokenizer.from_pretrained("google/t5-large-ssm-nq")


/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
2023-03-25 03:48:07.275311: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-25 03:48:07.275740: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-25 03:48:07.275763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If y

28 January 1885


In [4]:
# Save model
import os

output_dir="./HoSoTAs/ClosedQA/"
path = os.path.join(output_dir, "model")
t5_qa_model.save_pretrained(path)
t5_tok.save_pretrained(path)

('./HoSoTAs/ClosedQA/model/tokenizer_config.json',
 './HoSoTAs/ClosedQA/model/special_tokens_map.json',
 './HoSoTAs/ClosedQA/model/spiece.model',
 './HoSoTAs/ClosedQA/model/added_tokens.json',
 './HoSoTAs/ClosedQA/model/tokenizer.json')

In [3]:
input_ids = t5_tok("When was Lincoln born?", return_tensors="pt").input_ids
gen_output = t5_qa_model.generate(input_ids)[0]

print(t5_tok.decode(gen_output, skip_special_tokens=True))

/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1809


In [8]:
# BoolQ
t5_qa_model_boolq = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-boolq")
t5_tok_boolq = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-boolq")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [15]:
from datasets import load_dataset, DownloadMode, load_metric
train_df = load_dataset("boolq", split="train")

Found cached dataset boolq (/home/agupt135/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)


In [27]:
train_df

Dataset({
    features: ['question', 'answer', 'passage'],
    num_rows: 9427
})

In [36]:
import pandas as pd
df = pd.DataFrame({
    'input_text': train_df['question'],
    'target_text' : train_df['answer'],
    'passage' : train_df['passage'],
})
df

,input_text,target_text,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,False,As with other games in The Elder Scrolls serie...
...,...,...,...
9422,is a us district court a federal court,True,The United States district courts are the gene...
9423,can a tenant get a restraining order against a...,True,"If a landlord is found to be retaliating, he o..."
9424,is the golden state warriors in the playoffs,True,The Warriors went into the 2018 playoffs as th...
9425,downton abbey will there be a season 7,False,Downton Abbey is a British period drama televi...


In [37]:
# Fine Tune BoolQ
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    learning_rate=5e-3,
    output_dir="./HoSoTAs/BoolQ/",
    evaluation_strategy="steps",
    per_device_train_batch_size=180,
    num_train_epochs = 3,
    predict_with_generate=True,
    logging_steps=1000,  # set to 1000 for full training
    save_steps=500,  # set to 500 for full training
    eval_steps=8000,  # set to 8000 for full training
    warmup_steps=2000,  # set to 2000 for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, # True if GPU
    
)

trainer = Seq2SeqTrainer(
    model = t5_qa_model_boolq,
    args = training_args,
    train_dataset = train_df
)
t5_qa_model_finetuned_boolq = trainer.train()

/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


IndexError: Invalid key: 9406 is out of bounds for size 0

In [13]:
input_ids = t5_tok_boolq("is earth flat?", return_tensors="pt").input_ids
gen_output = t5_qa_model_boolq.generate(input_ids)[0]

print(t5_tok.decode(gen_output, skip_special_tokens=True))

yes


In [38]:
# Save model
import os

output_dir="./HoSoTAs/BoolQ/"
path = os.path.join(output_dir, "model")
t5_qa_model_boolq.save_pretrained(path)
t5_tok_boolq.save_pretrained(path)

('./HoSoTAs/BoolQ/model/tokenizer_config.json',
 './HoSoTAs/BoolQ/model/special_tokens_map.json',
 './HoSoTAs/BoolQ/model/spiece.model',
 './HoSoTAs/BoolQ/model/added_tokens.json',
 './HoSoTAs/BoolQ/model/tokenizer.json')

In [9]:
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

output_dir="./FAMNet/HoSoTAs/en-de-new"
path = os.path.join(output_dir, "model_files")
trained_tokenizer = T5Tokenizer.from_pretrained(path)
trained_model = T5ForConditionalGeneration.from_pretrained(path)

text = 'Resumption of the session'
text1 = 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.'
inference(trained_model=trained_model, trained_tokenizer=trained_tokenizer, text=text1)

[{'generated_text': 'Ich erkläre die am Freitag, den 17. Dezember 1999 unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen und wünsche Ihnen nochmals ein glückliches neues Jahr in der Hoffnung,'}]


In [4]:
def inference(trained_model, trained_tokenizer, text):
    generator = pipeline(
        "text2text-generation",
        model=trained_model,
        tokenizer=trained_tokenizer,
        device=0 if torch.cuda.is_available() else -1, # use GPU if available
    )

    input_text = text # input text for generation
    generated_text = generator(input_text, max_length=50)
    print(generated_text)